In [2]:
import pandas as pd
df = pd.read_csv(r'data/teg-all-data-long.csv')
print(df['TEGNum'].dtype)
teg_df = df.groupby(['Pl', 'TEG', 'TEGNum'], as_index=False)[['Sc', 'Stableford', 'GrossVP', 'NetVP']].sum().sort_values(by=['Pl','TEGNum'])
teg_df.to_clipboard(index=False)


int64


In [39]:
import dash
from dash import dcc, html, dash_table
import pandas as pd

# Sample data
data = {'Player': ['Stuart Neumann', 'Gregg Williams', 'David Mullin', 'Jon Baker', 'Alex Baker'],
        'R1': [39, 34, 33, 34, 29],
        'R2': [35, 30, 34, 32, 32],
        'R3': [39, 37, 30, 33, 33],
        'R4': [43, 42, 45, 32, 33],
        'Total': [156, 143, 142, 131, 127]}

df = pd.DataFrame(data)

# Add a Rank column based on the Total column (descending order)
df['Rank'] = df['Total'].rank(ascending=False, method='min').astype(int)

# Reorder the columns, making 'Rank' the first column
df = df[['Rank', 'Player', 'R1', 'R2', 'R3', 'R4', 'Total']]

# Initialize the Dash app
app = dash.Dash(__name__)

# List of columns to make grey
grey_columns = ['R1', 'R2', 'R3', 'R4']

# Create a conditional list for grey columns dynamically
style_cell_conditional = [
    {
        'if': {'column_id': 'Player'},
        'textAlign': 'left',
        'paddingLeft': '15px'  # Add left padding to the first column
    },
    {
        'if': {'column_id': 'Total'},
        'fontWeight': 'bold',    # Bold the last column (Total)
    },
    {
        'if': {'column_id': 'Rank'},
        'fontSize': '12px',      # Smaller font for the Rank column
        'width': '60px',         # Narrow column width
        'fontWeight': 'normal'   # No bold for Rank column
    }
]

# Append the conditional styling for grey color to the grey columns
style_cell_conditional += [{'if': {'column_id': col}, 'color': 'grey'} for col in grey_columns]

app.layout = html.Div([
    dash_table.DataTable(
        data=df.to_dict('records'),
        # Columns with a blank header for Rank
        columns=[{'name': '' if col == 'Rank' else col, 'id': col} for col in df.columns],
        style_cell={
            'fontFamily': 'Arial',
            'fontSize': '14px',
            'textAlign': 'center',
            'height': '50px'  # Increase the row height
        },
        style_cell_conditional=style_cell_conditional,
        # Header styling with thick black border below for all headers
        style_header={
            'backgroundColor': 'white',
            'fontWeight': 'bold',
            'borderBottom': '3px solid black',  # Thick black border for all headers
            'borderTop': 'none',
            'borderLeft': 'none',
            'borderRight': 'none'
        },
        # Add faint grey horizontal border below each row
        style_data={
            'borderBottom': '1px solid lightgrey',  # Faint grey horizontal border
            'borderTop': 'none',
            'borderLeft': 'none',
            'borderRight': 'none'
        },
        style_as_list_view=True,
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)


In [41]:
data = {'Player': ['Stuart Neumann', 'Gregg Williams', 'David Mullin', 'Jon Baker', 'Alex Baker'],
        'R1': [39, 34, 33, 34, 29],
        'R2': [35, 30, 34, 32, 32],
        'R3': [39, 37, 30, 33, 33],
        'R4': [43, 42, 45, 32, 33],
        'Total': [156, 143, 142, 131, 127]}

df = pd.DataFrame(data)



print("Original DataFrame:")
print(df)

# Method 1: Reset index and drop
df_reset = df.reset_index(drop=True)
print("\nDataFrame after reset_index(drop=True):")
print(df_reset)

# Method 2: Set index to None
df.index = None
print("\nOriginal DataFrame after setting index to None:")
print(df)

Original DataFrame:
           Player  R1  R2  R3  R4  Total
0  Stuart Neumann  39  35  39  43    156
1  Gregg Williams  34  30  37  42    143
2    David Mullin  33  34  30  45    142
3       Jon Baker  34  32  33  32    131
4      Alex Baker  29  32  33  33    127

DataFrame after reset_index(drop=True):
           Player  R1  R2  R3  R4  Total
0  Stuart Neumann  39  35  39  43    156
1  Gregg Williams  34  30  37  42    143
2    David Mullin  33  34  30  45    142
3       Jon Baker  34  32  33  32    131
4      Alex Baker  29  32  33  33    127


TypeError: Index(...) must be called with a collection of some kind, None was passed

In [ ]:
# Sample DataFrame
data = {'Player': ['Stuart Neumann', 'Gregg Williams', 'David Mullin', 'Jon Baker', 'Alex Baker'],
        'R1': [39, 34, 33, 34, 29],
        'R2': [35, 30, 34, 32, 32],
        'R3': [39, 37, 30, 33, 33],
        'R4': [43, 42, 45, 32, 33],
        'Total': [156, 143, 142, 131, 127]}

df = pd.DataFrame(data)
df['Rank'] = df['Total'].rank(ascending=False, method='min').astype(int)
df = df[['Rank', 'Player', 'R1', 'R2', 'R3', 'R4', 'Total']]


In [49]:
import pandas as pd

df = pd.read_csv(r'data/teg-all-data-long.csv')
measures = ['Sc', 'GrossVP', 'NetVP', 'Stableford']
group_rd = ['Pl', 'TEGNum', 'TEG', 'Round']
round_df = df.groupby(group_rd, as_index=False)[measures].sum().sort_values(by=['Pl','TEGNum','Round'])

chosen_teg = 'TEG 16'
leaderboard_df = round_df[round_df['TEG' = chosen_teg]]

def create_leaderboard(leaderboard_df):
    # Create pivot table and calculate total in one step
    jacketboard_pivot = leaderboard_df.pivot_table(
        index='Pl', 
        columns='Round', 
        values='GrossVP', 
        aggfunc='sum', 
        fill_value=0
    ).assign(Total=lambda x: x.sum(axis=1)).sort_values('Total', ascending=True)

    # Reset index and calculate rank
    jacketboard_pivot = jacketboard_pivot.reset_index()
    jacketboard_pivot['Rank'] = jacketboard_pivot['Total'].rank(method='min', ascending=True).astype(int)

    # Handle tied ranks
    duplicated_scores = jacketboard_pivot['Total'].duplicated(keep=False)
    jacketboard_pivot.loc[duplicated_scores, 'Rank'] = jacketboard_pivot.loc[duplicated_scores, 'Rank'].astype(str) + '='

    # Reorder columns
    columns = ['Rank', 'Pl'] + [col for col in jacketboard_pivot.columns if col not in ['Rank', 'Pl']]
    jacketboard_pivot = jacketboard_pivot[columns]

    return jacketboard_pivot



chosen_teg = 'TEG 16'
leaderboard_df = round_df[round_df['TEG'] == chosen_teg]

result = create_leaderboard(leaderboard_df)
print(result.to_string(index=False))



Rank Pl  1  2  3  4  Total
   1 GW 18 22 15 11     66
   2 DM 23 22 26 11     82
   3 JB 21 23 25 24     93
   4 SN 26 30 26 23    105
   5 AB 42 34 34 38    148


C:\Users\JBA33\AppData\Local\Temp\ipykernel_28316\2986125347.py:26: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.



In [14]:
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials
from math import floor

# Import the player name lookup function from player_utils.py
from player_utils import get_player_name

# Google Sheets API setup
SCOPE = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
CREDS_PATH = r"credentials\maps-1489139675490-41bee944be4e.json"  # Path to your service account JSON file

# Authorize the client
creds = Credentials.from_service_account_file(CREDS_PATH, scopes=SCOPE)
client = gspread.authorize(creds)

# Specify the Google Sheet and worksheet
SHEET_NAME = "TEG Round Input"  # Name of your Google Sheet
WORKSHEET_NAME = "Scores"  # Name of the specific worksheet (tab)

# Open the Google Sheet and select the worksheet
sheet = client.open(SHEET_NAME).worksheet(WORKSHEET_NAME)

# Fetch all records (rows) from the worksheet
data = sheet.get_all_records()

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Define non-player columns (fixed columns)
id_vars = ['TEGNum', 'Round', 'Hole', 'Par', 'SI']  # Columns to keep

# Convert player columns to a standard list for cleaner printing
player_columns = list(df.columns[5:])

# Melt the DataFrame to convert wide to long format
long_df = pd.melt(df, id_vars=id_vars, value_vars=player_columns, var_name='Pl', value_name='Score')

# Convert the 'Score' column to numeric, if necessary
long_df['Score'] = pd.to_numeric(long_df['Score'], errors='coerce')

# Exclude rows where Score is either NaN (blank) or 0
long_df = long_df.dropna(subset=['Score'])  # Remove rows where Score is NaN
long_df = long_df[long_df['Score'] != 0]    # Remove rows where Score is 0

# Ensure there are 18 holes per round per player
long_df = long_df.groupby(['TEGNum', 'Round', 'Pl']).filter(lambda x: len(x) == 18)

# Load the handicaps.csv file
hc_lookup = pd.read_csv('data/handicaps.csv')

# Melt the wide-format handicaps table to long format
hc_long = pd.melt(hc_lookup, id_vars='TEG', var_name='Pl', value_name='HC')

# Remove rows where 'HC' is missing or zero
hc_long = hc_long.dropna(subset=['HC'])
hc_long = hc_long[hc_long['HC'] != 0]

# Define the transformations for the long format DataFrame
def apply_transformations(long_df):
    # Score becomes Sc
    long_df['Sc'] = long_df['Score']
    long_df.drop('Score', axis=1, inplace=True)

    # Par becomes PAR
    long_df['PAR'] = long_df['Par']
    long_df.drop('Par', axis=1, inplace=True)

    # TEG = 'TEG ' + TEGNum
    long_df['TEG'] = 'TEG ' + long_df['TEGNum'].astype(str)

    # Merge the handicaps with the long_df data
    long_df = long_df.merge(hc_long, on=['TEG', 'Pl'], how='left')

    # Handle NaN values by creating a new DataFrame without using inplace
    long_df['HC'] = long_df['HC'].fillna(0)

    # HoleID = combination of TEG, Round, Hole in format 'T00|R00|H00'
    long_df['HoleID'] = long_df.apply(lambda row: f"T{int(row['TEGNum']):02}|R{int(row['Round']):02}|H{int(row['Hole']):02}", axis=1)

    # FrontBack = 'Front' if Hole < 10, otherwise 'Back'
    long_df['FrontBack'] = long_df['Hole'].apply(lambda hole: 'Front' if hole < 10 else 'Back')

    # Player = get player name from initials (Pl)
    long_df['Player'] = long_df['Pl'].apply(get_player_name)

    # HCStrokes = Excel formula equivalent =1*(MOD(HC,18)>=SI)+FLOOR(HC/18,1)
    long_df['HCStrokes'] = long_df.apply(lambda row: 1 * (row['HC'] % 18 >= row['SI']) + floor(row['HC'] / 18), axis=1)

    # GrossVP = Sc - PAR
    long_df['GrossVP'] = long_df['Sc'] - long_df['PAR']

    # NetVP = GrossVP - HCStrokes
    long_df['NetVP'] = long_df['GrossVP'] - long_df['HCStrokes']

    # Stableford = max(0, 2 - NetVP)
    long_df['Stableford'] = long_df['NetVP'].apply(lambda x: max(0, 2 - x))

    return long_df

# Apply transformations
transformed_df = apply_transformations(long_df)

# Display the transformed data for verification
print(transformed_df.head())

# After applying transformations, check combinations of HC, SI, and HCStrokes
def check_hc_strokes_combinations(transformed_df):
    # Select the relevant columns for the check
    hc_si_strokes_df = transformed_df[['HC', 'SI', 'HCStrokes']]
    
    # Drop duplicates to get unique combinations
    unique_combinations = hc_si_strokes_df.drop_duplicates()
    
    # Copy the unique combinations to clipboard
    unique_combinations.to_clipboard(index=False)
    
    print("Unique combinations of HC, SI, and HCStrokes have been copied to the clipboard.")

# Check the combinations in the transformed data and send to clipboard
#check_hc_strokes_combinations(transformed_df)

# After applying transformations, check combinations of HC, SI, and HCStrokes
def check_TRH_combinations(transformed_df):
    # Select the relevant columns for the check
    trh_df = transformed_df[['TEG', 'Round', 'Hole','HoleID']]
    
    # Drop duplicates to get unique combinations
    unique_combinations = trh_df.drop_duplicates()
    
    # Copy the unique combinations to clipboard
    unique_combinations.to_clipboard(index=False)
    
    print("Unique combinations of TRH have been copied to the clipboard.")

check_TRH_combinations(transformed_df)

   TEGNum  Round  Hole  SI  Pl  Sc  PAR     TEG    HC       HoleID FrontBack  \
0      50      1     1   8  AB   5    4  TEG 50  38.0  T50|R01|H01     Front   
1      50      1     2  14  AB   8    5  TEG 50  38.0  T50|R01|H02     Front   
2      50      1     3  18  AB   3    3  TEG 50  38.0  T50|R01|H03     Front   
3      50      1     4  12  AB   4    4  TEG 50  38.0  T50|R01|H04     Front   
4      50      1     5   2  AB   6    4  TEG 50  38.0  T50|R01|H05     Front   

       Player  HCStrokes  GrossVP  NetVP  Stableford  
0  Alex BAKER          2        1     -1           3  
1  Alex BAKER          2        3      1           1  
2  Alex BAKER          2        0     -2           4  
3  Alex BAKER          2        0     -2           4  
4  Alex BAKER          3        2     -1           3  
Unique combinations of TRH have been copied to the clipboard.


In [15]:
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials
from data_utilities import process_round_for_all_scores  # Import the renamed function

# Google Sheets API setup
SCOPE = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
CREDS_PATH = r"credentials\maps-1489139675490-41bee944be4e.json"  # Path to your service account JSON file

# Authorize the client
creds = Credentials.from_service_account_file(CREDS_PATH, scopes=SCOPE)
client = gspread.authorize(creds)

# Specify the Google Sheet and worksheet
SHEET_NAME = "TEG Round Input"  # Name of your Google Sheet
WORKSHEET_NAME = "Scores"  # Name of the specific worksheet (tab)

# Open the Google Sheet and select the worksheet
sheet = client.open(SHEET_NAME).worksheet(WORKSHEET_NAME)

# Fetch all records (rows) from the worksheet
data = sheet.get_all_records()

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Define non-player columns (fixed columns)
id_vars = ['TEGNum', 'Round', 'Hole', 'Par', 'SI']  # Columns to keep

# Convert player columns to a standard list for cleaner printing
player_columns = list(df.columns[5:])

# Melt the DataFrame to convert wide to long format
long_df = pd.melt(df, id_vars=id_vars, value_vars=player_columns, var_name='Pl', value_name='Score')

# Convert the 'Score' column to numeric, if necessary
long_df['Score'] = pd.to_numeric(long_df['Score'], errors='coerce')

# Exclude rows where Score is either NaN (blank) or 0
long_df = long_df.dropna(subset=['Score'])  # Remove rows where Score is NaN
long_df = long_df[long_df['Score'] != 0]    # Remove rows where Score is 0

# Ensure there are 18 holes per round per player
long_df = long_df.groupby(['TEGNum', 'Round', 'Pl']).filter(lambda x: len(x) == 18)

# Load the handicaps.csv file
hc_lookup = pd.read_csv('data/handicaps.csv')

# Melt the wide-format handicaps table to long format
hc_long = pd.melt(hc_lookup, id_vars='TEG', var_name='Pl', value_name='HC')

# Remove rows where 'HC' is missing or zero
hc_long = hc_long.dropna(subset=['HC'])
hc_long = hc_long[hc_long['HC'] != 0]

# Apply transformations from data_utilities.py
transformed_df = process_round_for_all_scores(long_df, hc_long)

# Display the transformed data for verification
print(transformed_df.head())


   TEGNum  Round  Hole  SI  Pl  Sc  PAR     TEG    HC       HoleID FrontBack  \
0      50      1     1   8  AB   5    4  TEG 50  38.0  T50|R01|H01     Front   
1      50      1     2  14  AB   8    5  TEG 50  38.0  T50|R01|H02     Front   
2      50      1     3  18  AB   3    3  TEG 50  38.0  T50|R01|H03     Front   
3      50      1     4  12  AB   4    4  TEG 50  38.0  T50|R01|H04     Front   
4      50      1     5   2  AB   6    4  TEG 50  38.0  T50|R01|H05     Front   

       Player  HCStrokes  GrossVP  NetVP  Stableford  
0  Alex BAKER          2        1     -1           3  
1  Alex BAKER          2        3      1           1  
2  Alex BAKER          2        0     -2           4  
3  Alex BAKER          2        0     -2           4  
4  Alex BAKER          3        2     -1           3  
